In [ ]:
from sklearn.metrics import precision_recall_curve
from sklearn.metrics import average_precision_score
from sklearn.metrics import average_precision_score, precision_score, roc_curve, auc,roc_auc_score,recall_score,f1_score, classification_report, confusion_matrix,accuracy_score
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import  precision_recall_curve,precision_recall_fscore_support
import itertools 
from scipy import interp
from itertools import cycle
from sklearn.utils.multiclass import unique_labels

In [ ]:

def average_precision(algo,y_test_f,y_score_f,n_classes_f):
    # Compute ROC curve and ROC area for each class
    y_test_f  = label_binarize(y_test_f,  classes=[j for j in range(n_classes_f)])
    y_score_f = label_binarize(y_score_f, classes=[j for j in range(n_classes_f)])
    # For each class
    precision = dict()
    recall = dict()
    average_precision = dict()
    for i in range(n_classes_f):
        precision[i], recall[i], _ = precision_recall_curve(y_test_f[:, i],y_score_f[:, i])
        average_precision[i] = average_precision_score(y_test_f[:, i], y_score_f[:, i])

    # A "micro-average": quantifying score on all classes jointly
    precision["micro"], recall["micro"], _ = precision_recall_curve(y_test_f.ravel(),y_score_f.ravel())
    average_precision["micro"] = average_precision_score(y_test_f, y_score_f, average="micro")
    print('Average precision score, micro-averaged over all classes: {0:0.2f}'
          .format(average_precision["micro"]))
    fig = plt.figure()
    plt.step(recall['micro'], precision['micro'], color='b', alpha=0.2,where='post')
    plt.fill_between(recall["micro"], precision["micro"], alpha=0.2, color='b')

    plt.xlabel('Recall')
    plt.ylabel('Precision')
    plt.ylim([0.0, 1.05])
    plt.xlim([0.0, 1.0])
    plt.title('Average precision score, micro-averaged over all classes: AP={0:0.2f}'.format(average_precision["micro"]))
    if n_classes_f==4:
      path='./classification/'
    else:
      path='./classification2/'
    fig.savefig(path+algo+'_average_precision.pdf')
    return average_precision["micro"]

In [ ]:
def plot_roc_auc(algo,y_test_f,y_score_f,n_classes_f):
    # Compute ROC curve and ROC area for each class
    y_test_f  = label_binarize(y_test_f,  classes=[j for j in range(n_classes_f)])
    y_score_f = label_binarize(y_score_f, classes=[j for j in range(n_classes_f)])
    
    fpr = dict()
    tpr = dict()
    roc_auc = dict()
    for i in range(n_classes_f):
        fpr[i], tpr[i], _ = roc_curve(y_test_f[:, i], y_score_f[:, i])
        roc_auc[i] = auc(fpr[i], tpr[i])

    # Compute micro-average ROC curve and ROC area
    fpr["micro"], tpr["micro"], _ = roc_curve(y_test_f.ravel(), y_score_f.ravel())
    roc_auc["micro"] = auc(fpr["micro"], tpr["micro"])
    
   
    lw = 2
    # Compute macro-average ROC curve and ROC area

    # First aggregate all false positive rates
    all_fpr = np.unique(np.concatenate([fpr[i] for i in range(n_classes_f)]))

    # Then interpolate all ROC curves at this points
    mean_tpr = np.zeros_like(all_fpr)
    for i in range(n_classes_f):
        mean_tpr += interp(all_fpr, fpr[i], tpr[i])

    # Finally average it and compute AUC
    mean_tpr /= n_classes_f

    fpr["macro"] = all_fpr
    tpr["macro"] = mean_tpr
    roc_auc["macro"] = auc(fpr["macro"], tpr["macro"])

    # Plot all ROC curves
    #plt.figure()
    fig = plt.figure()
    plt.plot(fpr["micro"], tpr["micro"],
             label='micro-average ROC curve (area = {0:0.2f})'
                   ''.format(roc_auc["micro"]),
             color='deeppink', linestyle=':', linewidth=4)

    plt.plot(fpr["macro"], tpr["macro"],
             label='macro-average ROC curve (area = {0:0.2f})'
                   ''.format(roc_auc["macro"]),
             color='navy', linestyle=':', linewidth=4)

    colors = cycle(['aqua', 'darkorange', 'cornflowerblue'])
    for i, color in zip(range(n_classes_f), colors):
        plt.plot(fpr[i], tpr[i], color=color, lw=lw,
                 label='ROC curve of class {0} (area = {1:0.2f})'
                 ''.format(i, roc_auc[i]))

    plt.plot([0, 1], [0, 1], 'k--', lw=lw)
    plt.xlim([0.0, 1.0])
    plt.ylim([0.0, 1.05])
    plt.xlabel('False Positive Rate')
    plt.ylabel('True Positive Rate')
    plt.title('Receiver operating characteristic to multi-class')
    plt.legend(loc="lower right")
    plt.show()
    
    if n_classes_f==4:
      path='./classification/'
    else:
      path='./classification2/'
    fig.savefig(path+algo+'_roc_auc.pdf')
    return roc_auc["macro"]

#def plot_confusion_matrix(y_true, y_pred, classes, normalize=False, title=None, cmap=plt.cm.Blues):
def plot_confusion_matrix1(algo,cm, y_true_f, y_pred_f, classes_f,normalize=False, title='Confusion matrix', cmap=plt.cm.Blues):
    """
    This function prints and plots the confusion matrix.
    Normalization can be applied by setting `normalize=True`.
    
    if not title:
        if normalize:
            title = 'Normalized confusion matrix'
        else:
            title = 'Confusion matrix, without normalization'
    """
    # Compute confusion matrix
    #cm = confusion_matrix(y_true_f, y_pred_f)
    # Only use the labels that appear in the data
    #y_true_f = np.array(y_true_f).astype(int)
    #y_pred_f = np.array(y_pred_f).astype(int)
    #classes_f = classes_f[unique_labels(y_true_f, y_pred_f)]
    if normalize:
        cm = cm.astype('float') / cm.sum(axis=1)[:, np.newaxis]
        print("Normalized confusion matrix")
    else:
        print('Confusion matrix, without normalization')

    print(cm)

    fig, ax = plt.subplots()
    im = ax.imshow(cm, interpolation='nearest', cmap=cmap)
    ax.figure.colorbar(im, ax=ax)
    # We want to show all ticks...
    ax.set(xticks=np.arange(cm.shape[1]),
           yticks=np.arange(cm.shape[0]),
           # ... and label them with the respective list entries
           xticklabels=classes_f, yticklabels=classes_f,
           #title=title,
           ylabel='True label',
           xlabel='Predicted label')

    # Rotate the tick labels and set their alignment.
    plt.setp(ax.get_xticklabels(), rotation=45, ha="right",
             rotation_mode="anchor")

    # Loop over data dimensions and create text annotations.
    fmt = '.2f' if normalize else 'd'
    thresh = cm.max() / 2.
    for i in range(cm.shape[0]):
        for j in range(cm.shape[1]):
            ax.text(j, i, format(cm[i, j], fmt),
                    ha="center", va="center",
                    color="white" if cm[i, j] > thresh else "black")
    fig.tight_layout()
    plt.ylabel('True label')
    plt.xlabel('Predicted label')
    #plt.tight_layout()
    fig.tight_layout()
    if len(classes_f)==4:
      path='./classification/'
    else:
      path='./classification2/'
    fig.savefig(path+algo+'_'+title+'_confusion_matrix.pdf')
    return ax
    
def plot_confusion_matrix(algo,cm, classes_f,normalize=False, title='Confusion matrix', cmap=plt.cm.Blues):
    """
    This function prints and plots the confusion matrix.
    Normalization can be applied by setting `normalize=True`.
    """
    if normalize:
        cm = cm.astype('float') / cm.sum(axis=1)[:, np.newaxis]
        print("Normalized confusion matrix")
    else:
        print('Confusion matrix, without normalization')

    print(cm)
    fig = plt.figure()
    plt.imshow(cm, interpolation='nearest', cmap=cmap)
    plt.title(title)
    plt.colorbar()
    tick_marks = np.arange(len(classes_f))
    plt.xticks(tick_marks, classes_f, rotation=45)
    plt.yticks(tick_marks, classes_f)

    fmt = '.2f' if normalize else 'd'
    thresh = cm.max() / 2.
    for i, j in itertools.product(range(cm.shape[0]), range(cm.shape[1])):
        plt.text(j, i, format(cm[i, j], fmt),
                 horizontalalignment="center",
                 color="white" if cm[i, j] > thresh else "black")

    plt.ylabel('True label')
    plt.xlabel('Predicted label')
    #plt.tight_layout()
    fig.tight_layout()
    if n_classes_f==4:
      path='./classification/'
    else:
      path='./classification2/'
    fig.savefig(path+algo+'_'+title+'_confusion_matrix.pdf')

# define function for plotting the results of algorithms with additional lag feature
def compute_plot_result_3(algo, time, y_true_f, y_pred_f,n_classess_f,target_namess_f,score_table):
    """compute, log and plot the performance for both training and test sets"""
    #labels=[i for i in range(n_classess_f)]
    
    stuff = precision_recall_fscore_support(y_true_f, y_pred_f)
    
    print(classification_report(y_true_f, y_pred_f, target_names=target_namess_f))
    print("Precision score: {}".format(precision_score(y_true_f,y_pred_f, average=None)))
    print("accuracy score: {}".format(accuracy_score(y_true_f,y_pred_f)))
    print("Recall score: {}".format(recall_score(y_true_f,y_pred_f, average=None)))
    print("F1 Score: {}".format(f1_score(y_true_f,y_pred_f, average=None)))
    
    print ("Precision is:",stuff[0][0])
    Precision= stuff[0][0]
    print ("Recall is:",stuff[1][0])
    Recall =stuff[1][0]
    print ("F1 is:",stuff[2][0])
    F1 =stuff[2][0]
     # compute the performance
    #r2_train = r2_score(y_train, pred_train)
    accuracy      = accuracy_score(y_true_f,y_pred_f)
    roc_auc       = plot_roc_auc(algo,y_true_f,y_pred_f,n_classess_f)
    average_prec  = average_precision(algo,y_true_f,y_pred_f,n_classess_f) 
    r2_test       = r2_score(y_true_f, y_pred_f)
    #mse_train = mean_squared_error(y_train, pred_train)
    mse_test      = mean_squared_error(y_true_f, y_pred_f)
    
    score_table.loc[algo,:] = time, accuracy, Precision, Recall, F1,roc_auc, average_prec, r2_test, mse_test
        
    #print(classification_report(y_true_f,y_pred_f))
    confusion_df = pd.DataFrame(confusion_matrix(y_true_f,y_pred_f),
             columns=["Predicted Class " + str(class_name) for class_name in target_namess_f],
             index = ["Class " + str(class_name) for class_name in target_namess_f])

    #print(confusion_df)
    # Compute confusion matrix
    cnf_matrix = confusion_matrix(y_true_f,y_pred_f)
    np.set_printoptions(precision=2)

    # Plot non-normalized confusion matrix
    plt.figure()
    plot_confusion_matrix1(algo,cnf_matrix, y_true_f, y_pred_f, classes_f=target_namess_f, title='without normalization')

    # Plot normalized confusion matrix
    plt.figure()
    plot_confusion_matrix1(algo,cnf_matrix, y_true_f, y_pred_f, classes_f=target_namess_f, normalize=True, title='Normalized')
    plt.show()

# Start

In [ ]:
# initialize a score table to log the performance of various algorithms
index = ['LogisticRegression','LinearDiscriminantAnalysis','KNeighborsClassifier','DecisionTreeClassifier','GaussianNB',
         'SVC','RandomForestClassifier','XGBClassifier','AdaBoostClassifier','MLPClassifier','Deep Learning']
score_table_2  = pd.DataFrame(index = index, columns= ['Time','Accuracy', 'Precision', 'Recall', 'F1','roc_auc_score','average_precision_score', 'r2_test', 'mse_test'])

score_table_tot = pd.DataFrame(index = index, columns= ['Time','Accuracy', 'Precision', 'Recall', 'F1','roc_auc_score','average_precision_score', 'r2_test', 'mse_test'])

score_table_3 = pd.DataFrame(index = index,columns= ['Parameters','Best parameters','Accuracy'])

# Logistic Regression models

In [ ]:
import time

start_time = time.clock()

# LASSO model with the new lag feature

# initialize a Lasso model with default parameters
linreg_2= Pipeline([('classifier',LogisticRegression())])

# grid search of parameter alpha to improve lasso model
parameters = [{'classifier__penalty': ['l1', 'l2'], 'classifier__C' : [1e2,1e5,1e6,1e8,1e9]}]
linreg_2_cv = GridSearchCV(linreg_2, parameters, n_jobs=-1, scoring= 'accuracy',cv=10)
linreg_2_cv.fit(X_train, y_train)
#results.append(linreg_2_cv)
print('Best parameters:', linreg_2_cv.best_params_)
print('Corresponding accuracy score:', linreg_2_cv.best_score_)
mean= linreg_2_cv.cv_results_['mean_test_score'][linreg_2_cv.best_index_]
std = linreg_2_cv.cv_results_['std_test_score'][linreg_2_cv.best_index_]
print("%f (%f)" % (mean, std))


algo='LogisticRegression'
score_table_3.loc[algo,:] =parameters,linreg_2_cv.best_params_,(mean, std)

In [ ]:
# simple LinearRegression model with the new lag feature
pen =linreg_2_cv.best_params_['classifier__penalty']
cp  =linreg_2_cv.best_params_['classifier__C']

linreg_2 = Pipeline([('scaler', StandardScaler()),('LogisticRegression',LogisticRegression(C=cp,penalty=pen))])
linreg_2.fit(X_train, y_train)
pred_train = linreg_2.predict(X_train)
pred_test = linreg_2.predict(X_test)

#feature_rank = pd.DataFrame({'feature':X_train.columns, 'coefficient':linreg_2.named_steps.linreg.coef_})
#print(feature_rank.sort_values('coefficient',ascending=False))
time1 = time.clock() - start_time
compute_plot_result_3('LogisticRegression', time1, y_test, pred_test,n_classes,target_names,score_table_2)#y_true,y_pred


# Model comparison

In [ ]:
import matplotlib.pyplot as plt

def highlight_max(data, color='green'):
    '''
    highlight the maximum in a Series or DataFrame
    '''
    attr = 'background-color: {}'.format(color)
    if data.ndim == 1:  # Series from .apply(axis=0) or axis=1
        is_max = data == data.max()
        return [attr if v else '' for v in is_max]
    else:  # from .apply(axis=None)
        is_max = data == data.max().max()
        return pd.DataFrame(np.where(is_max, attr, ''), index=data.index, columns=data.columns)

def highlight_min(data, color='red'):
    '''
    highlight the maximum in a Series or DataFrame
    '''
    attr = 'background-color: {}'.format(color)
    if data.ndim == 1:  # Series from .apply(axis=0) or axis=1
        is_min = data == data.min()
        return [attr if v else '' for v in is_min]
    else:  # from .apply(axis=None)
        is_min = data == data.min().min()
        return pd.DataFrame(np.where(is_min, attr, ''), index=data.index, columns=data.columns)


def draw_plot(data, edge_color, fill_color):
    bp = ax.boxplot(data, patch_artist=True)
    ax.set_xticklabels(data.index,rotation=90)
    for element in ['boxes', 'whiskers', 'fliers', 'means', 'medians', 'caps']:
        plt.setp(bp[element], color=edge_color)

    for patch in bp['boxes']:
        patch.set(facecolor=fill_color)       

fig, ax = plt.subplots()
draw_plot(score_table_3["Accuracy"], 'red', 'tan')
draw_plot(score_table_2[['Accuracy','Precision']], 'blue', 'cyan')
plt.show()

In [ ]:
score_table_2

In [ ]:
score_table_2.style.apply(highlight_max).\
apply(highlight_min, color='darkorange',  subset=['Accuracy','Precision','Recall',
                                                  'F1','r2_test','mse_test']).set_precision(4)

#score_table_2.style.format({lambda x: "{:.5f}".format(x)})